# Final Function for downloading the latex files

In [1]:
import arxiv
from pandas import DataFrame
import numpy as np
import pandas as pd
import os
import re

In [6]:
meta_data = pd.read_csv("meta_data_new.csv") 

In [4]:
meta_data.head()

,entry_id,updated,published,title,authors,summary,comment,journal_ref,doi,primary_category,categories,links,pdf_url,_raw,download_id,download,conference,uncommented_data
0,http://arxiv.org/abs/0907.0809v1,2009-07-04 22:34:25+00:00,2009-07-04 22:34:25+00:00,Learning as Search Optimization: Approximate L...,"[arxiv.Result.Author('Hal Daumé III'), arxiv.R...","Mappings to structured output spaces (strings,...",NaN,ICML 2005,NaN,cs.LG,"['cs.LG', 'cs.CL']",[arxiv.Result.Link('http://arxiv.org/abs/0907....,http://arxiv.org/pdf/0907.0809v1,"{'id': 'http://arxiv.org/abs/0907.0809v1', 'gu...",0907.0809v1,907.08090,ICML,\documentclass{article}\n\usepackage[accepted]...
1,http://arxiv.org/abs/1305.1704v1,2013-05-08 03:21:31+00:00,2013-05-08 03:21:31+00:00,The Extended Parameter Filter,"[arxiv.Result.Author('Yusuf Erol'), arxiv.Resu...","The parameters of temporal models, such as dyn...",NaN,ICML 2013,NaN,stat.ML,"['stat.ML', 'cs.AI']",[arxiv.Result.Link('http://arxiv.org/abs/1305....,http://arxiv.org/pdf/1305.1704v1,"{'id': 'http://arxiv.org/abs/1305.1704v1', 'gu...",1305.1704v1,1305.17040,ICML,\n\documentclass{article}\n\n\usepackage[tight...
2,http://arxiv.org/abs/1602.01783v2,2016-06-16 16:38:45+00:00,2016-02-04 18:38:41+00:00,Asynchronous Methods for Deep Reinforcement Le...,"[arxiv.Result.Author('Volodymyr Mnih'), arxiv....",We propose a conceptually simple and lightweig...,NaN,ICML 2016,NaN,cs.LG,['cs.LG'],[arxiv.Result.Link('http://arxiv.org/abs/1602....,http://arxiv.org/pdf/1602.01783v2,"{'id': 'http://arxiv.org/abs/1602.01783v2', 'g...",1602.01783v2,1602.01783,ICML,\documentclass{article} \usepackage{times}\n\u...
3,http://arxiv.org/abs/1603.03629v2,2016-06-10 19:04:59+00:00,2016-03-11 14:02:20+00:00,Square Root Graphical Models: Multivariate Gen...,"[arxiv.Result.Author('David I. Inouye'), arxiv...","We develop Square Root Graphical Models (SQR),...",NaN,ICML 2016,NaN,stat.ML,['stat.ML'],[arxiv.Result.Link('http://arxiv.org/abs/1603....,http://arxiv.org/pdf/1603.03629v2,"{'id': 'http://arxiv.org/abs/1603.03629v2', 'g...",1603.03629v2,1603.03629,ICML,\documentclass{article}\n\pdfoutput=1\n\usepac...
4,http://arxiv.org/abs/1609.00288v2,2017-09-01 08:18:33+00:00,2016-09-01 15:49:43+00:00,A Unified View of Multi-Label Performance Meas...,"[arxiv.Result.Author('Xi-Zhu Wu'), arxiv.Resul...",Multi-label classification deals with the prob...,NaN,ICML 2017,NaN,cs.LG,['cs.LG'],[arxiv.Result.Link('http://arxiv.org/abs/1609....,http://arxiv.org/pdf/1609.00288v2,"{'id': 'http://arxiv.org/abs/1609.00288v2', 'g...",1609.00288v2,1609.00288,ICML,\pdfoutput=1\n\documentclass{article}\n\n\n\us...


##### We this line we drop every value of the meta dataset to avoid keeping the same search result

In [5]:
meta_data = meta_data.iloc[0:0]
meta_data

,entry_id,updated,published,title,authors,summary,comment,journal_ref,doi,primary_category,categories,links,pdf_url,_raw,download_id,download,conference,uncommented_data


In [5]:
def search_query_conference(conference_name,meta_data):
    print('start')
    results = arxiv.Search(query='jr:'+ conference_name, max_results=200).get()
    results_as_dicts = [vars(r) for r in results]
    df = DataFrame(data=results_as_dicts)
    df['conference']=conference_name
    
    #working on journal_ref to exclude workshops
    nan_value = float("NaN")
    df.replace("", nan_value, inplace=True)
    df.dropna(subset = ["journal_ref"], inplace=True)
    
    df2 = (df.drop(df[df['journal_ref'].str.contains('workshop') |
                 df['journal_ref'].str.contains('Workshop')].index))
    
    #working on entry_id to extract the id    
    df2['download_id']=df2['entry_id'].str.strip().str.split("/").str[-1]
    df2['download']=df2['download_id'].str.strip().str[:-2]
    keys = df2["download_id"].tolist()
    good_keys=[]
    for i in range(len(keys)):
        if "." in keys[i]:
            good_keys.append(keys[i])
            good_keys=[x for x in good_keys if x not in list(meta_data['download_id'])]          
    print(good_keys)
    try:
        os.makedirs("./papers_new/"+conference_name)
    except FileExistsError:
        # directory already exists
        pass
   
    print(list(meta_data['download_id']))
    for key in good_keys[:10]:
        paper = next(arxiv.Search(id_list=[key]).get())
        # Download the archive to a specified directory with a custom filename.
        paper.download_source(dirpath="./papers_new/"+conference_name, filename=key+'.tar.gz')
        print('paper.download_source ', key,'.tar.gz' )
        
    df3=df2[df2["download_id"].isin(good_keys[:10]) == True]
    result=pd.concat([meta_data,df3]).reset_index(drop=True)
    result=result.drop_duplicates(subset=['entry_id'])
    
    return result  

In [6]:
# ADDED THIS FOR CODE READABILITY #

In [7]:
conferences = ["ICML", "ICLR", "IJCAI", "ECAI", "ECML PKDD"] #small conferences in artificial intelligence
for i in range(3):
    for conf in conferences:
        print("Downloading conference: ", conf)
        meta_data=search_query_conference(conf,meta_data)

start
['0907.0809v1', '1305.1704v1', '1602.01783v2', '1603.03629v2', '1609.00288v2', '1711.09576v4', '1711.11279v5', '1801.04062v4', '1801.09819v5', '1802.10217v3', '1803.05428v5', '1806.08734v3', '1810.10126v7', '1901.08469v3', '1902.03251v2', '1902.08077v2', '1905.03673v2', '1905.05313v1', '1905.06125v1', '1908.09345v2', '1910.04817v4', '1910.08013v3', '2002.02886v4', '2002.03229v2', '2002.03967v3', '2002.07290v2', '2002.09615v2', '2003.07845v2', '2006.14744v3', '2006.16540v2', '2007.01738v1', '2007.02010v1', '2011.10680v3', '2105.04297v2', '2105.12022v1', '2105.13695v1', '2106.04815v2', '2106.07689v1', '2106.10410v1', '2106.12491v1', '1712.05055v2', '1802.01421v4', '1904.03626v3', '2002.04745v2', '2002.05056v2', '2006.14218v2', '2006.14755v2', '2004.05988v5', '1206.4627v1', '1805.08196v1', '1811.00164v3', '1901.09087v2', '1903.00450v3', '1905.02685v5', '1907.03211v4', '1910.00189v2', '1912.03517v3', '2002.09339v2', '2002.11544v1', '2002.12414v2', '2006.12087v2', '2006.12598v2', '200

paper.download_source  1910.05744v3 .tar.gz
paper.download_source  0903.0471v1 .tar.gz
paper.download_source  0903.1136v1 .tar.gz
paper.download_source  0903.1152v1 .tar.gz
paper.download_source  1805.02917v1 .tar.gz
paper.download_source  1911.09539v2 .tar.gz
paper.download_source  0707.0652v1 .tar.gz
paper.download_source  1504.00854v1 .tar.gz
paper.download_source  1910.13530v2 .tar.gz
paper.download_source  2006.16365v1 .tar.gz
start
['2107.01988v1', '1902.05804v2', '1905.00609v2', '2006.09533v1', '1208.0984v1', '1312.5419v3', '1905.02005v2', '1007.0832v1', '2006.09186v1', '1404.5065v1', '1911.12899v1', '1809.01062v1', '1407.4832v1', '2004.03019v2', '1909.06108v1', '1904.05381v1', '1906.03105v5', '2007.00480v1', '1612.05440v3', '1708.06131v1', '2004.06353v1', '2105.03841v1', '1109.6804v1', '1704.06687v1', '1908.00698v2', '1706.09865v2', '1804.06620v3', '1807.04098v1', '1908.02641v2', '1612.03412v2', '1812.01735v1', '2004.04256v1', '1908.02571v3', '1906.11889v5', '2006.09174v1', '09

paper.download_source  1811.01900v3 .tar.gz
paper.download_source  1811.09620v2 .tar.gz
paper.download_source  1902.00177v2 .tar.gz
paper.download_source  1902.07151v2 .tar.gz
paper.download_source  1904.07785v1 .tar.gz
paper.download_source  1905.09591v1 .tar.gz
paper.download_source  1905.10227v3 .tar.gz
paper.download_source  1905.13209v4 .tar.gz
paper.download_source  1907.02957v2 .tar.gz
paper.download_source  1908.05699v4 .tar.gz
start
['1612.06530v2', '1705.08664v1', '1804.08139v1', '1805.02874v1', '1805.08524v1', '1805.09991v1', '1805.10421v2', '1903.04959v1', '1903.09109v2', '1905.10022v1', '1905.10048v4', '1905.13672v2', '1906.00781v1', '1906.04362v1', '1906.09320v1', '1906.09954v1', '2006.10932v1', '2007.08024v1', '2008.09657v3', '2008.13578v4', '2011.11499v1', '2012.06845v1', '2012.06850v1', '2101.00074v1', '2103.04485v1', '2103.07769v2', '2104.14769v1', '2105.03640v2', '0905.3763v1', '0905.3766v1', '0905.3769v1', '1304.5970v1', '1712.07745v1', '1806.02308v1', '1806.11298v2

paper.download_source  1910.04817v4 .tar.gz
paper.download_source  1910.08013v3 .tar.gz
paper.download_source  2002.02886v4 .tar.gz
paper.download_source  2002.03229v2 .tar.gz
paper.download_source  2002.03967v3 .tar.gz
paper.download_source  2002.07290v2 .tar.gz
paper.download_source  2002.09615v2 .tar.gz
paper.download_source  2003.07845v2 .tar.gz
paper.download_source  2006.14744v3 .tar.gz
paper.download_source  2006.16540v2 .tar.gz
start
['2001.04678v2', '2001.05017v1', '2002.01365v2', '2002.08396v3', '2003.02389v1', '2003.10823v2', '2004.11714v1', '2006.14567v3', '2007.06007v3', '2011.03902v3', '2011.04583v3', '2101.06967v1', '2103.03905v2', '2103.04181v1', '2103.06503v1', '2105.07381v1', '2106.01901v1', '2107.00233v1', '1810.10108v1', '1906.00588v5', '1911.06317v4', '2001.09382v2', '1804.06216v2', '1912.10903v1', '1702.08811v3', '1707.03815v4', '1802.07896v4', '1803.07770v1', '1807.05031v6', '1808.07593v4', '1809.03008v3', '1901.00243v2', '1905.08101v1', '1906.00230v6', '1909.027

paper.download_source  2005.09276v2 .tar.gz
paper.download_source  1002.0573v1 .tar.gz
paper.download_source  1008.0660v1 .tar.gz
paper.download_source  1501.07256v1 .tar.gz
paper.download_source  1801.01612v1 .tar.gz
paper.download_source  1801.03597v1 .tar.gz
paper.download_source  1801.05459v1 .tar.gz
paper.download_source  1901.08361v3 .tar.gz
paper.download_source  1901.08552v1 .tar.gz
paper.download_source  1909.02815v1 .tar.gz
start
['2004.06353v1', '2105.03841v1', '1109.6804v1', '1704.06687v1', '1908.00698v2', '1706.09865v2', '1804.06620v3', '1807.04098v1', '1908.02641v2', '1612.03412v2', '1812.01735v1', '2004.04256v1', '1908.02571v3', '1906.11889v5', '2006.09174v1', '0906.4258v1', '1807.00462v1', '1006.3424v2', '1304.5350v3', '0706.3435v1', '1811.02215v1', '1808.02207v2', '1909.07873v1']
['0907.0809v1', '1305.1704v1', '1602.01783v2', '1603.03629v2', '1609.00288v2', '1711.09576v4', '1711.11279v5', '1801.04062v4', '1801.09819v5', '1802.10217v3', '1301.3224v5', '1412.6632v5', '16

In [8]:
meta_data.shape

(150, 18)

In [11]:
meta_data.to_csv(r'meta_data_new.csv', index = False)

In [12]:
import shutil
path_to_conferences = "./papers_new"
conferences = os.listdir(path_to_conferences)

if '.DS_Store' in conferences:
    conferences.remove('.DS_Store')

In [13]:
for conference in conferences:
    print(conference)
    path_to_each_conferences=path_to_conferences+"/"+conference
    papers = os.listdir(path_to_each_conferences)
    print('----------------------------------')

    for paper in papers:
        partitioned_string = paper.rpartition('.tar')
        papername = partitioned_string[0]
        print("=========== ",papername)
        
        path_to_each_conference_paper=path_to_each_conferences+"/"+paper
        """print(path_to_each_conference_paper)"""
        if path_to_each_conference_paper.endswith("tar.gz"):
            dest="./"+"unzipped_papers_new"+"/"+conference+"/"+papername
            print(dest)
            
            shutil.unpack_archive(path_to_each_conference_paper, dest)                

ECAI
----------------------------------
===========  0707.0652v1
./unzipped_papers_new/ECAI/0707.0652v1
===========  0903.0471v1
./unzipped_papers_new/ECAI/0903.0471v1
===========  0903.1136v1
./unzipped_papers_new/ECAI/0903.1136v1
===========  0903.1152v1
./unzipped_papers_new/ECAI/0903.1152v1
===========  1002.0573v1
./unzipped_papers_new/ECAI/1002.0573v1
===========  1008.0660v1
./unzipped_papers_new/ECAI/1008.0660v1
===========  1308.4526v5
./unzipped_papers_new/ECAI/1308.4526v5
===========  1501.07256v1
./unzipped_papers_new/ECAI/1501.07256v1
===========  1504.00854v1
./unzipped_papers_new/ECAI/1504.00854v1
===========  1801.01612v1
./unzipped_papers_new/ECAI/1801.01612v1
===========  1801.03597v1
./unzipped_papers_new/ECAI/1801.03597v1
===========  1801.05459v1
./unzipped_papers_new/ECAI/1801.05459v1
===========  1805.02917v1
./unzipped_papers_new/ECAI/1805.02917v1
===========  1901.08361v3
./unzipped_papers_new/ECAI/1901.08361v3
===========  1901.08552v1
./unzipped_papers_new/EC

===========  1606.09594v1
./unzipped_papers_new/IJCAI/1606.09594v1
===========  1609.07028v2
./unzipped_papers_new/IJCAI/1609.07028v2
===========  1610.08462v1
./unzipped_papers_new/IJCAI/1610.08462v1
===========  1612.06530v2
./unzipped_papers_new/IJCAI/1612.06530v2
===========  1705.08664v1
./unzipped_papers_new/IJCAI/1705.08664v1
===========  1804.08139v1
./unzipped_papers_new/IJCAI/1804.08139v1
===========  1805.02874v1
./unzipped_papers_new/IJCAI/1805.02874v1
===========  1805.08524v1
./unzipped_papers_new/IJCAI/1805.08524v1
===========  1805.09991v1
./unzipped_papers_new/IJCAI/1805.09991v1
===========  1805.10421v2
./unzipped_papers_new/IJCAI/1805.10421v2
===========  1903.04959v1
./unzipped_papers_new/IJCAI/1903.04959v1
===========  1903.09109v2
./unzipped_papers_new/IJCAI/1903.09109v2
===========  1905.10022v1
./unzipped_papers_new/IJCAI/1905.10022v1
===========  1905.10048v4
./unzipped_papers_new/IJCAI/1905.10048v4
===========  1905.13672v2
./unzipped_papers_new/IJCAI/1905.136